In [138]:
import json
import pandas as pd
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim.models
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models import KeyedVectors
import gensim.downloader as api
import zipfile
import sys
import requests, io
import re 
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Необходимо создать класс, позволяющий тестировать качество работы моделей

In [139]:
def preprocess(text:str, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if re.match(r'(\d.|\d)', lemma) is None:
                if lemma not in stop_words:
                    preprocessed_text.append(lemma)
    return preprocessed_text

def read_json(path: str):
    file = open(path)
    data = json.load(file)
    return pd.DataFrame(data)

punctuation_marks = ['!', ',', '(', ')', ';', ':', '-', '?', '.', '..', '...', "\"", "/", "\`\`", "»", "«" ]
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [315]:
class ModelResearcher:
    def __init__(self):
        self.model = None
    
    def preprocess_and_save(self, data_df: pd.DataFrame, path: str, text_field='text') -> pd.DataFrame:
        # for preprocessing dataset. Use it only in critical cases cause it's too slow on big datasets
        data_df['preprocessed_' + text_field] = data_df.apply(lambda row: preprocess(row[text_field], punctuation_marks, stop_words, morph), axis=1)
        data_df_preprocessed = data_df.copy()
        data_df_preprocessed = data_df_preprocessed.drop(columns=[text_field], axis=1)
        data_df_preprocessed.to_json(path)
        return data_df_preprocessed
    
    def preprocess_and_save_pairs(self, data_df: pd.DataFrame, path: str, text_field_1, text_field_2) -> pd.DataFrame:
        data_df['preprocessed_' + text_field_1] = data_df.apply(lambda row: preprocess(row[text_field_1], punctuation_marks, stop_words, morph), axis=1)
        data_df['preprocessed_' + text_field_2] = data_df.apply(lambda row: preprocess(row[text_field_2], punctuation_marks, stop_words, morph), axis=1)
        data_df_preprocessed = data_df.copy()
        data_df_preprocessed = data_df_preprocessed.drop(columns=[text_field_1, text_field_2], axis=1)
        data_df_preprocessed.to_json(path)
        return data_df_preprocessed
    
    def train(self, data_df: pd.DataFrame, model="w2v"):            
        if model == "w2v":
            train_part = data_df['preprocessed_texts']
            self.model = gensim.models.Word2Vec(sentences=train_part, min_count=5, vector_size=50, epochs=5)
        elif model == "fast_text":
            train_part = data_df['preprocessed_texts'].tolist()
            self.model = gensim.models.FastText(vector_size=50, min_count=5)
            self.model.build_vocab(corpus_iterable=train_part)
            self.model.train(corpus_iterable=train_part, total_examples=len(train_part), epochs=5)
        return
    
    def predict_sentences_similarity(self, sentences_1: pd.Series, sentences_2: pd.Series):
        if sentences_1.size != sentences_2.size:
            return None
        else:
            if self.model is not None:
                sentences_sim = np.zeros(sentences_1.size)
                sz = sentences_1.size
#                 print(f'index_to_key: {self.model.wv.index_to_key}')
                for i in range(sz): 
                    sentences_1_words = [w for w in sentences_1[i] if w in self.model.wv.index_to_key]
                    sentences_2_words = [w for w in sentences_2[i] if w in self.model.wv.index_to_key]
                    sim = self.model.wv.n_similarity(sentences_1_words, sentences_2_words)
                    sentences_sim[i] = sim
                
                return sentences_sim
            else:
                return None
            
    def calc_f1_score(self, sim_match, sim_unmatch, match_threshold):
        (TP, FP, FN, TN) = (0, 0, 0, 0)
        for i in range(sim_match.size):
            # у всех модели из данного списка должен быть sim_match[i] >= match_threshold
            if sim_match[i] >= match_threshold: 
                TP += 1
            else:
                FN += 1

        for i in range(sim_unmatch.size):
            if sim_unmatch[i] >= match_threshold: 
                FP += 1
            else:
                TN += 1

        return round(float(2*TP / (2*TP + FP + FN)), 3)

In [46]:
# data_df = read_json('./preprocessed_documents.json')
match_df = read_json('./match.json')
unmatch_df = read_json('./unmatch.json')

In [47]:
match_df['preprocessed_text_rp'] = match_df.apply(lambda row: preprocess(row['text_rp'], punctuation_marks, stop_words, morph), axis=1)
match_df['preprocessed_text_proj'] = match_df.apply(lambda row: preprocess(row['text_proj'], punctuation_marks, stop_words, morph), axis=1)

unmatch_df['preprocessed_text_rp'] = unmatch_df.apply(lambda row: preprocess(row['text_rp'], punctuation_marks, stop_words, morph), axis=1)
unmatch_df['preprocessed_text_proj'] = unmatch_df.apply(lambda row: preprocess(row['text_proj'], punctuation_marks, stop_words, morph), axis=1)

In [48]:
modelResearcher = ModelResearcher()
# data_df = modelResearcher.preprocess_and_save(data_df, './preprocessed_documents.json') !!! very much time
modelResearcher.train(data_df, model="w2v")

In [54]:
match_predict = modelResearcher.predict_sentences_similarity(match_df['preprocessed_text_rp'], match_df['preprocessed_text_proj'])
unmatch_predict = modelResearcher.predict_sentences_similarity(unmatch_df['preprocessed_text_rp'], unmatch_df['preprocessed_text_proj'])
print(f'F1-score: {modelResearcher.calc_f1_score(match_predict, unmatch_predict, 0.78)}')


F1-score: 0.762


In [56]:
match_df.drop('score', inplace=True, axis=1, errors='ignore')
match_df.insert(loc=4, column='score', value=match_predict) 
match_df

,id_rp,id_proj,name_rp,name_proj,score,text_rp,text_proj,need_match,preprocessed_text_rp,preprocessed_text_proj
0,1000,15,АЛГОРИТМЫ БЕСПИЛОТНОГО ТРАНСПОРТА,Разработка системы движения роя дронов в прост...,0.814201,Данный курс знакомит слушателей с основами упр...,Разработка системы движения роя дронов в прост...,True,"[данный, курс, знакомить, слушатель, основа, у...","[разработка, система, движение, рой, дрон, про..."
1,1005,40,АНАЛИЗ ДАННЫХ В ИСКУССТВЕННОМ ИНТЕЛЛЕКТЕ,Роботизированная система машинного творчества ...,0.781342,Освещаемые в курсе теоретические и прикладные ...,Роботизированная система машинного творчества ...,True,"[освещать, курс, теоретический, прикладной, во...","[роботизированный, система, машинный, творчест..."
2,1009,4,РАЗРАБОТКА ПРИЛОЖЕНИЙ В РАСПРЕДЕЛЕННОЙ СРЕДЕ,Разработка модулей обработки данных для библио...,0.710863,Дисциплина посвящена изучению основных принцип...,Разработка модулей обработки данных для библио...,True,"[дисциплина, посвятить, изучение, основный, пр...","[разработка, модуль, обработка, данные, библио..."
3,1011,14,Тестирование и сопровождение программного обес...,Тестирование и документирование стенда виртуал...,0.830194,"Цели дисциплины охватывают ряд направлений, св...",Тестирование и документирование стенда виртуал...,True,"[цель, дисциплина, охватывать, ряд, направлени...","[тестирование, документирование, стенд, виртуа..."
4,1016,46,Интеллектуальные системы,Разработка интеллектуальных модулей обработки ...,0.799842,Рассматриваются основные понятия теории интелл...,Разработка интеллектуальных модулей обработки ...,True,"[рассматриваться, основной, понятие, теория, и...","[разработка, интеллектуальный, модуль, обработ..."
5,105,41,Системы управления манипуляторными роботами,Разработка подводного телеуправляемого робота ...,0.625624,Дисциплина включает изложение основ теории ман...,Разработка подводного телеуправляемого робота ...,True,"[дисциплина, включать, изложение, основа, теор...","[разработка, подводный, телеуправляемый, робот..."
6,1040,26,Управление и информатика в технических системах,Программно-аппаратное конфигурирование сети ла...,0.741461,В рамках данной дисциплины рассматриваются осн...,Программно-аппаратное конфигурирование сети ла...,True,"[рамка, дать, дисциплина, рассматриваться, осн...","[программно-аппаратный, конфигурирование, сеть..."
7,1278,9,Тестирование программного обеспечения,Тестирование конфигураций на платформах: Super...,0.784438,Дисциплина посвящена изучению теоретических ос...,Тестирование конфигураций на платформах: Super...,True,"[дисциплина, посвятить, изучение, теоретически...","[тестирование, конфигурация, платформа, superm..."
8,991,30,Машинное обучение,Разработка системы детектирования и идентифика...,0.864563,Данный курс знакомит слушателей с основными ме...,Разработка системы детектирования и идентифика...,True,"[данный, курс, знакомить, слушатель, основный,...","[разработка, система, детектирование, идентифи..."
9,985,8,Основы компьютерного дизайна,"Разработка ИС ""Расписание"" / ""Деканат"".",0.650390,В курсе рассматриваются свойства зрительного в...,"Разработка ИС ""Расписание"" / ""Деканат"". Информ...",True,"[курс, рассматриваться, свойство, зрительный, ...","[разработка, иса, ``, расписание, '', /, ``, д..."


In [57]:
unmatch_df.drop('score', inplace=True, axis=1, errors='ignore')
unmatch_df.insert(loc=4, column='score', value=unmatch_predict) 
unmatch_df

,id_rp,id_proj,name_rp,name_proj,score,text_rp,text_proj,need_match,preprocessed_text_rp,preprocessed_text_proj
0,2645,11,Компоненты электронной техники,Анализ динамики COVID-19 в России,0.360414,Рассмотрены основные свойства и характеристики...,Web-приложение для отображения динамики различ...,False,"[рассмотреть, основной, свойство, характеристи...","[web-приложение, отображение, динамика, различ..."
1,2649,16,Философия,Разработка системы вскрытия объектов на кадрах...,0.457881,"Философия – гуманитарная дисциплина, изучающая...",Разработка системы вскрытия объектов на кадрах...,False,"[философия, –, гуманитарный, дисциплина, изуча...","[разработка, система, вскрытие, объект, кадр, ..."
2,2646,44,Диэлектрические материалы и приборы,Разработка системы проверки доставки писем на ...,0.295981,Рассматриваются современные представления о по...,Разработка системы проверки доставки писем на ...,False,"[рассматриваться, современный, представление, ...","[разработка, система, проверка, доставка, пись..."
3,2648,46,Конфликтология,Разработка интеллектуальных модулей обработки ...,0.514024,Дисциплина «Конфликтология» знакомит с основны...,Разработка интеллектуальных модулей обработки ...,False,"[дисциплина, «, конфликтология, », знакомить, ...","[разработка, интеллектуальный, модуль, обработ..."
4,2650,21,Теоретические основы высокочастотной электроте...,Развитие кабинетов Юридических и Физических лиц,0.128008,Рассматривается распространение электромагнитн...,В рамках проекта развития кабинетов Юридически...,False,"[рассматриваться, распространение, электромагн...","[рамка, проект, развитие, кабинет, юридический..."
5,4833,2,Социология,Разработка платформы сервисов личного кабинета...,0.480749,Курс нацелен на формирование у студентов знани...,Разработка платформы сервисов личного кабинета...,False,"[курс, нацелить, формирование, студент, знание...","[разработка, платформа, сервис, личный, кабине..."
6,4833,25,Социология,Android-приложение предобработки данных 3d-рек...,0.405736,Курс нацелен на формирование у студентов знани...,Android-приложение предобработки данных 3d-рек...,False,"[курс, нацелить, формирование, студент, знание...","[android-приложение, предобработка, данные, ви..."
7,1552,29,Методы и инструменты управления рисками органи...,Разработка программного комплекса для оценки в...,0.767502,"В данной дисциплине изучаются основы, термины ...",Разработка программного комплекса для оценки в...,False,"[дать, дисциплина, изучаться, основа, термин, ...","[разработка, программный, комплекс, оценка, ва..."
8,1557,30,Бережливые технологии в менеджменте качества,Разработка системы детектирования и идентифика...,0.742337,Целью освоения дисциплины «Бережливые технолог...,Разработка системы детектирования и идентифика...,False,"[цель, освоение, дисциплина, «, бережливый, те...","[разработка, система, детектирование, идентифи..."
9,1558,31,Оценка соответствия и аудит,Определение состояния автомобиля используя видео,0.558951,Дисциплина предназначена для изучения основ оц...,Определение состояния автомобиля используя вид...,False,"[дисциплина, предназначить, изучение, основа, ...","[определение, состояние, автомобиль, использов..."


## Проделаем то же с FastText

In [58]:
data_df = read_json('./preprocessed_documents.json')
match_df = read_json('./match.json')
unmatch_df = read_json('./unmatch.json')
match_df['preprocessed_text_rp'] = match_df.apply(lambda row: preprocess(row['text_rp'], punctuation_marks, stop_words, morph), axis=1)
match_df['preprocessed_text_proj'] = match_df.apply(lambda row: preprocess(row['text_proj'], punctuation_marks, stop_words, morph), axis=1)

unmatch_df['preprocessed_text_rp'] = unmatch_df.apply(lambda row: preprocess(row['text_rp'], punctuation_marks, stop_words, morph), axis=1)
unmatch_df['preprocessed_text_proj'] = unmatch_df.apply(lambda row: preprocess(row['text_proj'], punctuation_marks, stop_words, morph), axis=1)

In [59]:
modelResearcher = ModelResearcher()
# data_df = modelResearcher.preprocess_and_save(data_df, './preprocessed_documents.json') !!! very much time
modelResearcher.train(data_df, model="fast_text")

In [65]:
match_predict_ft = modelResearcher.predict_sentences_similarity(match_df['preprocessed_text_rp'], match_df['preprocessed_text_proj'])
unmatch_predict_ft = modelResearcher.predict_sentences_similarity(unmatch_df['preprocessed_text_rp'], unmatch_df['preprocessed_text_proj'])
print(f'F1-score: {modelResearcher.calc_f1_score(match_predict_ft, unmatch_predict_ft, 0.78)}')

F1-score: 0.75


In [61]:
match_df.drop('score', inplace=True, axis=1, errors='ignore')
match_df.insert(loc=4, column='score', value=match_predict_ft) 
match_df

,id_rp,id_proj,name_rp,name_proj,score,text_rp,text_proj,need_match,preprocessed_text_rp,preprocessed_text_proj
0,1000,15,АЛГОРИТМЫ БЕСПИЛОТНОГО ТРАНСПОРТА,Разработка системы движения роя дронов в прост...,0.881327,Данный курс знакомит слушателей с основами упр...,Разработка системы движения роя дронов в прост...,True,"[данный, курс, знакомить, слушатель, основа, у...","[разработка, система, движение, рой, дрон, про..."
1,1005,40,АНАЛИЗ ДАННЫХ В ИСКУССТВЕННОМ ИНТЕЛЛЕКТЕ,Роботизированная система машинного творчества ...,0.881194,Освещаемые в курсе теоретические и прикладные ...,Роботизированная система машинного творчества ...,True,"[освещать, курс, теоретический, прикладной, во...","[роботизированный, система, машинный, творчест..."
2,1009,4,РАЗРАБОТКА ПРИЛОЖЕНИЙ В РАСПРЕДЕЛЕННОЙ СРЕДЕ,Разработка модулей обработки данных для библио...,0.775276,Дисциплина посвящена изучению основных принцип...,Разработка модулей обработки данных для библио...,True,"[дисциплина, посвятить, изучение, основный, пр...","[разработка, модуль, обработка, данные, библио..."
3,1011,14,Тестирование и сопровождение программного обес...,Тестирование и документирование стенда виртуал...,0.874233,"Цели дисциплины охватывают ряд направлений, св...",Тестирование и документирование стенда виртуал...,True,"[цель, дисциплина, охватывать, ряд, направлени...","[тестирование, документирование, стенд, виртуа..."
4,1016,46,Интеллектуальные системы,Разработка интеллектуальных модулей обработки ...,0.838396,Рассматриваются основные понятия теории интелл...,Разработка интеллектуальных модулей обработки ...,True,"[рассматриваться, основной, понятие, теория, и...","[разработка, интеллектуальный, модуль, обработ..."
5,105,41,Системы управления манипуляторными роботами,Разработка подводного телеуправляемого робота ...,0.851550,Дисциплина включает изложение основ теории ман...,Разработка подводного телеуправляемого робота ...,True,"[дисциплина, включать, изложение, основа, теор...","[разработка, подводный, телеуправляемый, робот..."
6,1040,26,Управление и информатика в технических системах,Программно-аппаратное конфигурирование сети ла...,0.842299,В рамках данной дисциплины рассматриваются осн...,Программно-аппаратное конфигурирование сети ла...,True,"[рамка, дать, дисциплина, рассматриваться, осн...","[программно-аппаратный, конфигурирование, сеть..."
7,1278,9,Тестирование программного обеспечения,Тестирование конфигураций на платформах: Super...,0.833364,Дисциплина посвящена изучению теоретических ос...,Тестирование конфигураций на платформах: Super...,True,"[дисциплина, посвятить, изучение, теоретически...","[тестирование, конфигурация, платформа, superm..."
8,991,30,Машинное обучение,Разработка системы детектирования и идентифика...,0.859232,Данный курс знакомит слушателей с основными ме...,Разработка системы детектирования и идентифика...,True,"[данный, курс, знакомить, слушатель, основный,...","[разработка, система, детектирование, идентифи..."
9,985,8,Основы компьютерного дизайна,"Разработка ИС ""Расписание"" / ""Деканат"".",0.653468,В курсе рассматриваются свойства зрительного в...,"Разработка ИС ""Расписание"" / ""Деканат"". Информ...",True,"[курс, рассматриваться, свойство, зрительный, ...","[разработка, иса, ``, расписание, '', /, ``, д..."


In [316]:
match_df.drop('score', inplace=True, axis=1, errors='ignore')
unmatch_df.insert(loc=4, column='score', value=unmatch_predict_ft) 
unmatch_df

ValueError: cannot insert score, already exists

In [317]:
df = read_json('./dataset.json')

In [318]:
df

,id_rp,id_proj,name_rp,name_proj,text_rp,text_proj,need_match
0,1000,15,АЛГОРИТМЫ БЕСПИЛОТНОГО ТРАНСПОРТА,Разработка системы движения роя дронов в прост...,Данный курс знакомит слушателей с основами упр...,Разработка системы движения роя дронов в прост...,True
1,1005,40,АНАЛИЗ ДАННЫХ В ИСКУССТВЕННОМ ИНТЕЛЛЕКТЕ,Роботизированная система машинного творчества ...,Освещаемые в курсе теоретические и прикладные ...,Роботизированная система машинного творчества ...,True
2,1009,4,РАЗРАБОТКА ПРИЛОЖЕНИЙ В РАСПРЕДЕЛЕННОЙ СРЕДЕ,Разработка модулей обработки данных для библио...,Дисциплина посвящена изучению основных принцип...,Разработка модулей обработки данных для библио...,True
3,1011,14,Тестирование и сопровождение программного обес...,Тестирование и документирование стенда виртуал...,"Цели дисциплины охватывают ряд направлений, св...",Тестирование и документирование стенда виртуал...,True
4,1016,46,Интеллектуальные системы,Разработка интеллектуальных модулей обработки ...,Рассматриваются основные понятия теории интелл...,Разработка интеллектуальных модулей обработки ...,True
5,105,41,Системы управления манипуляторными роботами,Разработка подводного телеуправляемого робота ...,Дисциплина включает изложение основ теории ман...,Разработка подводного телеуправляемого робота ...,True
6,1040,26,Управление и информатика в технических системах,Программно-аппаратное конфигурирование сети ла...,В рамках данной дисциплины рассматриваются осн...,Программно-аппаратное конфигурирование сети ла...,True
7,1278,9,Тестирование программного обеспечения,Тестирование конфигураций на платформах: Super...,Дисциплина посвящена изучению теоретических ос...,Тестирование конфигураций на платформах: Super...,True
8,991,30,Машинное обучение,Разработка системы детектирования и идентифика...,Данный курс знакомит слушателей с основными ме...,Разработка системы детектирования и идентифика...,True
9,985,8,Основы компьютерного дизайна,"Разработка ИС ""Расписание"" / ""Деканат"".",В курсе рассматриваются свойства зрительного в...,"Разработка ИС ""Расписание"" / ""Деканат"". Информ...",True


In [319]:
# разбиваем датасет на 2 части 
df_train = pd.concat([pd.DataFrame(df[0:17]), pd.DataFrame(df[30:])], axis=0)
df_test = pd.DataFrame(df[17:30])

print('df_train: {} of {}'.format(df_train['id_rp'].size, df['id_rp'].size))
print('df_test: {} of {}'.format(df_test['id_rp'].size, df['id_rp'].size))

df_train: 33 of 46
df_test: 13 of 46


In [320]:
modelResearcher = ModelResearcher()
# df_train = modelResearcher.preprpocess_and_save_pairs(df_train, 'train.json', 'text_rp', 'text_proj')
# df_test = preprpocess_and_save_pairs()
df_train = modelResearcher.preprocess_and_save_pairs(df_train, 'train.json', 'text_rp', 'text_proj')
df_test = modelResearcher.preprocess_and_save_pairs(df_test, 'test.json', 'text_rp', 'text_proj')

In [321]:
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_test

,id_rp,id_proj,name_rp,name_proj,need_match,preprocessed_text_rp,preprocessed_text_proj
0,1098,56,Обработка изображений и компьютерное зрение (...,Практика в проектах по автономному транспорту ...,True,"[рассматриваться, основной, задача, модель, ме...","[практика, проект, автономный, транспорт, ао, ..."
1,2195,38,Основы гидроакустики,Разработка системы звукоподводной связи на осн...,True,"[аннотация, дисциплина, основа, гидроакустик, ...","[разработка, система, звукоподводный, связь, о..."
2,5089,31,Глубокое обучение,Определение состояния автомобиля используя видео,True,"[глубокий, обучение, являться, передовой, обла...","[определение, состояние, автомобиль, использов..."
3,3524,57,Разработка защищенных программных систем,Стажировка в Digital Security по аудиту защищё...,True,"[дисциплина, посвятить, изучение, современный,...","[стажировка, digital, security, аудит, защищён..."
4,3514,26,Технологии автоматизации процесса разработки п...,Программно-аппаратное конфигурирование сети ла...,True,"[курс, рассматриваться, вопрос, организация, п...","[программно-аппаратный, конфигурирование, сеть..."
5,1447,42,Моделирование нелинейных динамических систем,Перспективные сенсоры на основе теории динамич...,True,"[дисциплина, направить, ознакомление, студент,...","[перспективный, сенсор, основа, теория, динами..."
6,1916,41,Разработка программного обеспечения для робот...,Разработка подводного телеуправляемого робота ...,True,"[цель, изучение, курс, состоять, формирование,...","[разработка, подводный, телеуправляемый, робот..."
7,2645,11,Компоненты электронной техники,Анализ динамики COVID-19 в России,False,"[рассмотреть, основной, свойство, характеристи...","[web-приложение, отображение, динамика, различ..."
8,2649,16,Философия,Разработка системы вскрытия объектов на кадрах...,False,"[философия, –, гуманитарный, дисциплина, изуча...","[разработка, система, вскрытие, объект, кадр, ..."
9,2646,44,Диэлектрические материалы и приборы,Разработка системы проверки доставки писем на ...,False,"[рассматриваться, современный, представление, ...","[разработка, система, проверка, доставка, пись..."


In [322]:
df_train_fit = pd.DataFrame()
df_train_fit['preprocessed_texts'] = pd.concat([df_train['preprocessed_text_rp'], df_train['preprocessed_text_proj']], axis=0)

In [323]:
modelResearcher.train(df_train_fit, model="w2v")

In [324]:
result_w2v = modelResearcher.predict_sentences_similarity(df_test['preprocessed_text_rp'], df_test['preprocessed_text_proj'])

In [325]:
result_w2v

array([0.99993408, 0.99992198, 0.99996465, 0.9999463 , 0.99997091,
       0.99996436, 0.99994236, 0.99995846, 0.99995959, 0.99995172,
       0.99994981, 0.99994826, 0.99993157])

In [326]:
df_test

,id_rp,id_proj,name_rp,name_proj,need_match,preprocessed_text_rp,preprocessed_text_proj
0,1098,56,Обработка изображений и компьютерное зрение (...,Практика в проектах по автономному транспорту ...,True,"[рассматриваться, основной, задача, модель, ме...","[практика, проект, автономный, транспорт, ао, ..."
1,2195,38,Основы гидроакустики,Разработка системы звукоподводной связи на осн...,True,"[аннотация, дисциплина, основа, гидроакустик, ...","[разработка, система, звукоподводный, связь, о..."
2,5089,31,Глубокое обучение,Определение состояния автомобиля используя видео,True,"[глубокий, обучение, являться, передовой, обла...","[определение, состояние, автомобиль, использов..."
3,3524,57,Разработка защищенных программных систем,Стажировка в Digital Security по аудиту защищё...,True,"[дисциплина, посвятить, изучение, современный,...","[стажировка, digital, security, аудит, защищён..."
4,3514,26,Технологии автоматизации процесса разработки п...,Программно-аппаратное конфигурирование сети ла...,True,"[курс, рассматриваться, вопрос, организация, п...","[программно-аппаратный, конфигурирование, сеть..."
5,1447,42,Моделирование нелинейных динамических систем,Перспективные сенсоры на основе теории динамич...,True,"[дисциплина, направить, ознакомление, студент,...","[перспективный, сенсор, основа, теория, динами..."
6,1916,41,Разработка программного обеспечения для робот...,Разработка подводного телеуправляемого робота ...,True,"[цель, изучение, курс, состоять, формирование,...","[разработка, подводный, телеуправляемый, робот..."
7,2645,11,Компоненты электронной техники,Анализ динамики COVID-19 в России,False,"[рассмотреть, основной, свойство, характеристи...","[web-приложение, отображение, динамика, различ..."
8,2649,16,Философия,Разработка системы вскрытия объектов на кадрах...,False,"[философия, –, гуманитарный, дисциплина, изуча...","[разработка, система, вскрытие, объект, кадр, ..."
9,2646,44,Диэлектрические материалы и приборы,Разработка системы проверки доставки писем на ...,False,"[рассматриваться, современный, представление, ...","[разработка, система, проверка, доставка, пись..."
